In [1]:
import pandas as pd
import pickle

In [2]:
#df = pd.read_csv(r'book1.results.csv')
with open(r'C:\Users\anark\Desktop\Capstone\SQLChatBot\tuple_results.pkl', 'rb') as f:
    results = pickle.load(f)

# Convert dictionary to DataFrame
df = pd.DataFrame(results)

print(df)

                                          HumanQuery  \
0  count the states which have elevations lower t...   
1         give me the number of rivers in california   
2                give me the states that border utah   
3                         how high is mount mckinley   
4           how high is the highest point in america   
5        how long is the longest river in california   
6                         how long is the ohio river   
7                    how many people lived in austin   

                                          Gold Query  \
0  select count(highlow.state_name) from highlow ...   
1  select count(river.river_name) from river wher...   
2  select border_info.border from border_info whe...   
3  select highlow.highest_elevation from highlow ...   
4  select max(highlow.highest_elevation) from hig...   
5  select river.length from river where river.tra...   
6  select river.length from river where river.riv...   
7  select city.population from city where city.

In [3]:
df.head()

,HumanQuery,Gold Query,Generated Query,Gold Result,Generated Result,Generated Answer
0,count the states which have elevations lower t...,select count(highlow.state_name) from highlow ...,SELECT COUNT(*) FROM highlow WHERE CAST(lowest...,"[(2,)]","[(2,)]",The above query is counting the number of reco...
1,give me the number of rivers in california,select count(river.river_name) from river wher...,SELECT COUNT(*) FROM river WHERE traverse = 'c...,"[(1,)]","[(1,)]",The query is counting the number of rows in th...
2,give me the states that border utah,select border_info.border from border_info whe...,SELECT DISTINCT state_name FROM border_info WH...,"[(wyoming,), (colorado,), (new mexico,), (ariz...","[(arizona,), (colorado,), (idaho,), (nevada,),...",The SELECT DISTINCT state_name query from the ...
3,how high is mount mckinley,select highlow.highest_elevation from highlow ...,SELECT mountain_altitude FROM mountain WHERE m...,"[(6194,)]","[(6194,)]",The query is selecting the mountain_altitude f...
4,how high is the highest point in america,select max(highlow.highest_elevation) from hig...,"SELECT MAX(mountain_altitude) AS ""highest_poin...","[(979,)]","[(6194,)]",This query returns the maximum (highest) value...


In [4]:
#print(df['Gold Result'][2])
data_from_sqlite = df['Generated Answer'][2]
print(data_from_sqlite)
# Convert list of tuples to list of strings
'''
converted_data = [item[0] for item in data_from_sqlite]
print(converted_data)
'''

The SELECT DISTINCT state_name query from the border_info table where border is 'utah' returns a list of states that border Utah, including Arizona, Colorado, Idaho, Nevada, New Mexico, and Wyoming.


'\nconverted_data = [item[0] for item in data_from_sqlite]\nprint(converted_data)\n'

In [5]:
df['Gold Result'] = df['Gold Result'].apply(lambda x: [item[0] for item in x])
df['Generated Result'] = df['Generated Result'].apply(lambda x: [item[0] for item in x])

In [6]:
print(df['Gold Result'])

0                                                  [2]
1                                                  [1]
2    [wyoming, colorado, new mexico, arizona, nevad...
3                                               [6194]
4                                                [979]
5                                               [2333]
6           [1569, 1569, 1569, 1569, 1569, 1569, 1569]
7                                             [345496]
Name: Gold Result, dtype: object


In [7]:
print(df['Generated Result'])

0                                                  [2]
1                                                  [1]
2    [arizona, colorado, idaho, nevada, new mexico,...
3                                               [6194]
4                                               [6194]
5                                               [2333]
6           [1569, 1569, 1569, 1569, 1569, 1569, 1569]
7                                             [345496]
Name: Generated Result, dtype: object


In [8]:
from collections import Counter

def check_lists_equal(generated_list, gold_list):
    
    try:
        gold_list = [int(i) for i in gold_list]
    except: 
        print("\n") 
    print(gold_list, "\n", generated_list, "\n", Counter(generated_list) == Counter(gold_list))
    return Counter(generated_list) == Counter(gold_list)

# Apply the function to each row
df['Equal'] = df.apply(lambda row: check_lists_equal(row['Generated Result'], row['Gold Result']), axis=1)


[2] 
 [2] 
 True
[1] 
 [1] 
 True


['wyoming', 'colorado', 'new mexico', 'arizona', 'nevada', 'idaho'] 
 ['arizona', 'colorado', 'idaho', 'nevada', 'new mexico', 'wyoming'] 
 True
[6194] 
 [6194] 
 True
[979] 
 [6194] 
 False
[2333] 
 [2333] 
 True
[1569, 1569, 1569, 1569, 1569, 1569, 1569] 
 [1569, 1569, 1569, 1569, 1569, 1569, 1569] 
 True
[345496] 
 [345496] 
 True


In [9]:
print(df['Equal'])

0     True
1     True
2     True
3     True
4    False
5     True
6     True
7     True
Name: Equal, dtype: bool


In [14]:
print(df['Generated Answer'][2])

The SELECT DISTINCT state_name query from the border_info table where border is 'utah' returns a list of states that border Utah, including Arizona, Colorado, Idaho, Nevada, New Mexico, and Wyoming.


In [17]:
s = df['Generated Answer'][2]
s = s.replace(",", "")
print(s)
words_list = s.split(' ')
print(words_list)

The SELECT DISTINCT state_name query from the border_info table where border is 'utah' returns a list of states that border Utah including Arizona Colorado Idaho Nevada New Mexico and Wyoming.
['The', 'SELECT', 'DISTINCT', 'state_name', 'query', 'from', 'the', 'border_info', 'table', 'where', 'border', 'is', "'utah'", 'returns', 'a', 'list', 'of', 'states', 'that', 'border', 'Utah', 'including', 'Arizona', 'Colorado', 'Idaho', 'Nevada', 'New', 'Mexico', 'and', 'Wyoming.']


In [44]:
def check_result_in_answer(Generated_list, generated_answer):
    
    try:
        Generated_list = [int(i) for i in Generated_list]
    except: 
        print("\n") 
    #print(gold_list, "\n", generated_answer)
    generated_answer = generated_answer.replace(",", "").replace(".","")
    generated_answer = generated_answer.lower()
    words_list = generated_answer.split(' ')
    y = True
    for item in Generated_list:
        try:
            item = item.lower()
            item_list = item.split(' ')
            for item in item_list:
                try:
                    x = str(item) in words_list
                    print(str(item))
                    print(x)
                    if x == False:
                        y = False
                except:
                    x = item in words_list
                    print(x)
                    if x == False:
                        y = False
        except:
            try:
                x = str(item) in words_list
                print(str(item))
                print(x)
                if x == False:
                    y = False
            except:
                x = item in words_list
                print(x)
                if x == False:
                    y = False

    print(Generated_list, "\n", words_list)
    return y

df['Result in Answer'] = df.apply(lambda row: check_result_in_answer(row['Generated Result'], row['Generated Answer']), axis=1)

2
False
[2] 
 ['the', 'above', 'query', 'is', 'counting', 'the', 'number', 'of', 'records', 'in', 'the', '"highlow"', 'table', 'where', 'the', 'value', 'in', 'the', '"lowest_elevation"', 'column', 'is', 'less', 'than', 'the', 'lowest', 'elevation', 'value', 'in', 'alabama', 'it', 'returns', 'a', 'single', 'row', 'with', 'the', 'count', 'of', 'those', 'records', '']
1
True
[1] 
 ['the', 'query', 'is', 'counting', 'the', 'number', 'of', 'rows', 'in', 'the', '"river"', 'table', 'where', 'the', '"traverse"', 'column', 'has', 'a', 'value', 'of', '"california"', 'the', 'result', 'of', 'the', 'query', 'is', 'a', 'single', 'row', 'with', 'the', 'count', 'of', 'matching', 'rows', 'indicated', 'by', 'the', 'value', 'of', '1']


arizona
True
colorado
True
idaho
True
nevada
True
new
True
mexico
True
wyoming
True
['arizona', 'colorado', 'idaho', 'nevada', 'new mexico', 'wyoming'] 
 ['the', 'select', 'distinct', 'state_name', 'query', 'from', 'the', 'border_info', 'table', 'where', 'border', 'is', "

In [45]:
print(df['Result in Answer'])

0    False
1     True
2     True
3     True
4     True
5     True
6     True
7     True
Name: Result in Answer, dtype: bool


In [53]:
execution_accuracy = df['Equal'].mean() * 100

print(execution_accuracy,'%')

combined_accuracy = (df[(df['Result in Answer'] == True) & (df['Equal'] == True)].shape[0] / df.shape[0]) * 100

print(combined_accuracy,'%')



87.5 %
75.0 %
